In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 컬럼 설명
- clientnum : 고객 번호
- Attrition_flag : 고객으로 남아있는 경우와 떠난 고객 **Target**
- Cumtomer_age : 고객의 나이
- Gender : 고객의 성별
- Dependent_count : 부양 가족수
- Education_level : 교육 수준
- Marital_status : 결혼 여부 
- income_Category : 수입 수준
- card_category : 카드 유형(파랑, 실버, 골드, 플래티넘)
- Months_on_book : 은행과 거래 기간 월 기준
- Total_Relationship_Count : 한 고객이 소유한 카드의 개수
- Months_Inactive_12_mon : 12개월 내 비활성화된 횟수
- Contacts_Count_12_mon : 12개월 내 접촉된 횟수
- Credit_Limit : 카드 사용 금액 제한 정도
- Total_Revolving_Bal : 부채를 지고 결제 금액을 다음 달로 이월하는 정도
- Avg_Open_To_Buy : 신용을 통해 구매 가능한 금액 12개월 평균
- Total_Amt_Chng_Q4_Q1 : 거래 금액의 변도 1분기 대비 4분기
- Total_Trans_Amt : 총 거래금액 최근 12개월 이내
- Total_Trans_Ct : 총 거래 수 최근 12개월 이내
- Total_Ct_Chng_Q4_Q1 : 거래 수의 변환 1분기 대비 4분기
- Avg_Utilization_Ratio : 평균 카드 이용률

In [4]:
url = "https://raw.githubusercontent.com/DataResolvere/Project/main/first_project/BankChurners.csv"
df_row = pd.read_csv(url)

df_row["target"] = df_row["Attrition_Flag"]
df_row = df_row.drop(columns=["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", \
                              "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",\
                              "Attrition_Flag"])
df_row

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,target
0,768805383,45,M,3,High School,Married,$60K - $80K,Blue,39,5,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,Existing Customer
1,818770008,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,Existing Customer
2,713982108,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,Existing Customer
3,769911858,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,Existing Customer
4,709106358,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,Existing Customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,50,M,2,Graduate,Single,$40K - $60K,Blue,40,3,...,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,Existing Customer
10123,710638233,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,4,...,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,Attrited Customer
10124,716506083,44,F,1,High School,Married,Less than $40K,Blue,36,5,...,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,Attrited Customer
10125,717406983,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,4,...,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,Attrited Customer


In [7]:
make_en = LabelEncoder()
df_encorder = df_row.copy()
sd = StandardScaler()

df_encorder["Gender"] = make_en.fit_transform(df_encorder["Gender"])
df_encorder["Education_Level"] = make_en.fit_transform(df_encorder["Education_Level"])
df_encorder["Marital_Status"] = make_en.fit_transform(df_encorder["Marital_Status"])
df_encorder["Income_Category"] = make_en.fit_transform(df_encorder["Income_Category"])
df_encorder["Card_Category"] = make_en.fit_transform(df_encorder["Card_Category"])
df_encorder["target"] = make_en.fit_transform(df_encorder["target"])

df_encorder['Months_on_book'] = sd.fit_transform(df_encorder[['Months_on_book']])
df_encorder['Total_Relationship_Count'] = sd.fit_transform(df_encorder[['Total_Relationship_Count']])
df_encorder['Months_Inactive_12_mon'] = sd.fit_transform(df_encorder[['Months_Inactive_12_mon']])
df_encorder['Contacts_Count_12_mon'] = sd.fit_transform(df_encorder[['Contacts_Count_12_mon']])
df_encorder['Credit_Limit'] = sd.fit_transform(df_encorder[['Credit_Limit']])
df_encorder['Total_Revolving_Bal'] = sd.fit_transform(df_encorder[['Total_Revolving_Bal']])
df_encorder['Avg_Open_To_Buy'] = sd.fit_transform(df_encorder[['Avg_Open_To_Buy']])
df_encorder['Total_Amt_Chng_Q4_Q1'] = sd.fit_transform(df_encorder[['Total_Amt_Chng_Q4_Q1']])
df_encorder['Total_Trans_Amt'] = sd.fit_transform(df_encorder[['Total_Trans_Amt']])
df_encorder['Total_Trans_Ct'] = sd.fit_transform(df_encorder[['Total_Trans_Ct']])
df_encorder['Total_Ct_Chng_Q4_Q1'] = sd.fit_transform(df_encorder[['Total_Ct_Chng_Q4_Q1']])
df_encorder['Avg_Utilization_Ratio'] = sd.fit_transform(df_encorder[['Avg_Utilization_Ratio']])

In [8]:
x = df_encorder.drop(columns="target")
y = df_encorder[["target"]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=13)

In [9]:
# Clustering
km = KMeans(n_clusters=10, random_state=13)
km_data = km.fit(x_train[['Customer_Age', 'Dependent_count', 'Education_Level', 'Income_Category']])
x_train['clustering'] = km_data.labels_
x_test['clustering'] = km_data.predict(x_test[['Customer_Age', 'Dependent_count', 'Education_Level', 'Income_Category']])

#PCA
pca = PCA(n_components=2, random_state=13)

pca_data_train = pca.fit_transform(x_train[['Credit_Limit', "Total_Revolving_Bal",
                                           "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1",
                                           "Total_Trans_Amt", "Total_Trans_Ct",
                                           "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio"]])
pca_data_test = pca.transform(x_test[['Credit_Limit', "Total_Revolving_Bal",
                                           "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1",
                                           "Total_Trans_Amt", "Total_Trans_Ct",
                                           "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio"]])

x_train["pca1"] = pca_data_train[:, 0]
x_train["pca2"] = pca_data_train[:, 1]

x_test["pca1"] = pca_data_test[:, 0]
x_test["pca2"] = pca_data_test[:, 1]

#OverSampling
smote = SMOTE(random_state=13)
x_train_over, y_train_over = smote.fit_resample(x_train, y_train)
print(y_train_over.value_counts())

C:\Users\xswer\miniconda3\envs\ds_study\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


target
0         6799
1         6799
dtype: int64


In [10]:
x_train_over.shape, y_train_over.shape, x_test.shape, y_test.shape

((13598, 23), (13598, 1), (2026, 23), (2026, 1))

In [12]:
def print_modeling(x_train, x_test, y_train, y_test):
    dc = DecisionTreeClassifier()
    rf = RandomForestClassifier()
    xb = XGBClassifier()
    lg = LGBMClassifier()
    
    acc = []
    
    
    #DecisionTreeClassifier
    params = {
        "max_depth" : [3, 5, 7, 9, 11],
        'random_state' : [13]
    }
    dc_grid_model = GridSearchCV(dc, param_grid=params, cv = 5, scoring='accuracy')
    dc_grid_model.fit(x_train, y_train)
    pred = dc_grid_model.predict(x_test)
    acc.append(accuracy_score(y_test, pred))
    
    dc_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True)).transpose()
    
        
    # RandomForest
    params = {
        "n_estimators" : [100, 500, 1000, 1500, 2000],
        "max_depth" : [3, 5, 7 ,9],
        "random_state" : [13]
    }
    rf_grid = GridSearchCV(rf, param_grid=params, cv=5, scoring="accuracy")
    rf_grid.fit(x_train, y_train)
    pred = rf_grid.predict(x_test)
    acc.append(accuracy_score(y_test, pred))
    rf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True)).transpose()
    
    # xgbcBoost
    params = {
        "n_estimators" : [100, 500, 1000, 1500, 2000],
        "max_depth" : [3, 5, 7 ,9],
        "random_state" : [13]
    }
    xb_grid = GridSearchCV(xb, param_grid=params, cv=5, scoring="accuracy")
    xb_grid.fit(x_train, y_train)
    pred = xb_grid.predict(x_test)
    acc.append(accuracy_score(y_test, pred))
    xb_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True)).transpose()
    
    # lgbm
    params = {
        "n_estimators" : [100, 500, 1000, 1500, 2000],
        'application' : ['binary'],
        "max_depth" : [3, 5, 7, 9],
        "random_state" : [13]
    }
    lg_grid = GridSearchCV(lg, param_grid=params, cv=5, scoring="accuracy")
    lg_grid.fit(x_train, y_train)
    pred = lg_grid.predict(x_test)
    acc.append(accuracy_score(y_test, pred))
    lg_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True)).transpose()
    
    
    df = pd.DataFrame(data={"accuracy" : acc})
    df.rename(index={0 : "Decision", 1 : 'Randomforest', 2 : "XGBC", 3 : 'LGBMC'}, inplace = True)
    return df, dc_report, rf_report, xb_report, lg_report

In [13]:
result, dc_confusion, rf_confusion, xb_confusion, lg_confusion  = print_modeling(x_train_over, x_test, y_train_over.values.ravel(), y_test.values.ravel())
result

,accuracy
Decision,0.923495
Randomforest,0.939289
XGBC,0.961500
LGBMC,0.962488


In [14]:
xb_confusion

,precision,recall,f1-score,support
0,0.892063,0.864615,0.878125,325.0000
1,0.974284,0.980012,0.977140,1701.0000
accuracy,0.961500,0.961500,0.961500,0.9615
macro avg,0.933174,0.922314,0.927632,2026.0000
weighted avg,0.961095,0.961500,0.961256,2026.0000


In [15]:
lg_confusion

,precision,recall,f1-score,support
0,0.897764,0.864615,0.880878,325.000000
1,0.974314,0.981188,0.977739,1701.000000
accuracy,0.962488,0.962488,0.962488,0.962488
macro avg,0.936039,0.922901,0.929308,2026.000000
weighted avg,0.962034,0.962488,0.962201,2026.000000
